In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *

In [0]:
df = spark.read.load('/FileStore/tables/googleplaystore.csv', format='csv', sep =',', header='true', escape='"', inferschema = 'true' )


In [0]:
df.count()

Out[8]: 10841

In [0]:
df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [0]:
df = df.drop("size", "Content Rating", "Last Updated", "Android Ver")

In [0]:
df.show(5)

+--------------------+--------------+------+-------+-----------+----+-----+--------------------+------------------+
|                 App|      Category|Rating|Reviews|   Installs|Type|Price|              Genres|       Current Ver|
+--------------------+--------------+------+-------+-----------+----+-----+--------------------+------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|    10,000+|Free|    0|        Art & Design|             1.0.0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|   500,000+|Free|    0|Art & Design;Pret...|             2.0.0|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5,000,000+|Free|    0|        Art & Design|             1.2.4|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50,000,000+|Free|    0|        Art & Design|Varies with device|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|   100,000+|Free|    0|Art & Design;Crea...|               1.1|
+--------------------+--------------+------+-------+-----------+----+---

In [0]:
df = df.drop("Current Ver")

In [0]:
df.show(5)

+--------------------+--------------+------+-------+-----------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|   Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+-----------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|    10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|   500,000+|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5,000,000+|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50,000,000+|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|   100,000+|Free|    0|Art & Design;Crea...|
+--------------------+--------------+------+-------+-----------+----+-----+--------------------+
only showing top 5 rows



In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [0]:
from pyspark.sql.functions import regexp_extract, col

df = df.withColumn("Reviews", col("Reviews").cast(IntegerType()))\
.withColumn("Installs", regexp_replace(col("Installs"),"[^0-9]",""))\
.withColumn("Installs", col("Installs").cast(IntegerType()))\
    .withColumn("Price", regexp_replace(col("Price"),"[$]",""))\
        .withColumn("Price",col("Price").cast(IntegerType()))

In [0]:
df.show(5)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 5 rows



In [0]:
df.createOrReplaceTempView("apps")

In [0]:
%sql
select * from apps
limit 5;

App,Category,Rating,Reviews,Installs,Type,Price,Genres
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,10000,Free,0,Art & Design
Coloring book moana,ART_AND_DESIGN,3.9,967,500000,Free,0,Art & Design;Pretend Play
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,5000000,Free,0,Art & Design
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,50000000,Free,0,Art & Design
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,100000,Free,0,Art & Design;Creativity


In [0]:
%sql
SELECT App,Type, SUM(Reviews) AS total_reviews
FROM apps
GROUP BY App, Type
ORDER BY total_reviews DESC
LIMIT 5;  


App,Type,total_reviews
Instagram,Free,266241989
WhatsApp Messenger,Free,207348304
Clash of Clans,Free,179558781
Messenger – Text and Video Chat for Free,Free,169932272
Subway Surfers,Free,166331958


In [0]:
%sql
SELECT App, SUM(Installs) AS total_install
from apps
GROUP BY App
ORDER BY total_install DESC
LIMIT 5;

App,total_install
Subway Surfers,6000000000
Google Photos,4000000000
Hangouts,4000000000
Instagram,4000000000
Google Drive,4000000000


In [0]:
%sql
SELECT Category,SUM(Installs) from apps
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

Category,sum(Installs)
GAME,35086024415
COMMUNICATION,32647276251
PRODUCTIVITY,14176091369
SOCIAL,14069867902
TOOLS,11452771915


In [0]:
%sql
SELECT App,Type, SUM(Price) FROM apps
WHERE Type = "Paid"
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 5;

App,Type,sum(Price)
I'm Rich - Trump Edition,Paid,400
I am Rich!,Paid,399
💎 I'm rich,Paid,399
I am Rich Plus,Paid,399
I AM RICH PRO PLUS,Paid,399


In [0]:
# File __path__
# /FileStore/tables/googleplaystore.csv